In [1]:
from sklearn.preprocessing import normalize
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
import json
import gensim


from pprint import pprint


In [2]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('/home/domyoung/DeepLearning/wordembedding/pretrained/w2v_vectors.bin', binary=True,unicode_errors='ignore')

In [24]:
import fastText
fT=fastText.FastText.load_model('/home/domyoung/Airi/20181119-cleaned-all-words-x300-ws5-min50-minn2-maxn-6-epoch2.bin')

In [4]:
import p_mean as pmean  

In [3]:
import p_mean_FT as pmeanFT

## hyperparameters

In [5]:
dim = 300

In [6]:
meanlist=['mean','p_mean_3','p_mean_2']


## make sent_matrix 

In [7]:
line_list=[]
with open("./santafe_category.txt") as f:
    i=0

    for line in f:
        if line not in line_list:
            line_list.append(line)

In [8]:
sent_matrix=np.empty((0,dim))
sent2idx={}
idx2sent={}

i=0

for line in line_list:
    values = np.zeros(dim)
    # indexing
    sent2idx[line]=i
    idx2sent[i]=line


    k=0
    if(" @ " in line):
        line=line[:line.index(" @ ")]
        #strip newline
        line = line.replace('\n','')
    words = line.split(' ')
    
    for word in words:
        #it is the page number
        if(word == "@"):
            break
        #add
        if(word in w2v):
            
            values+=w2v[word]
            
            k+=1

    if k > 0:
        sent_matrix=np.vstack((sent_matrix,pmean.z_norm(values)))
    else: 
        sent_matrix=np.vstack((sent_matrix,values))
    
    i+=1
   
    if(i%100000==0):
        print(datetime.datetime.now().isoformat())
sent_matrix = normalize(sent_matrix)

## make query vector 

In [15]:
query = "리모컨 키 배터리 교체 방법 알려줘"
query_vec =None

words=query.split(' ')
k=0
for word in words:
    #add
    if(word in w2v):
        values+=np.squeeze(normalize(w2v[word].reshape(1,-1)))
        #values+=w2v[word]
        k+=1
if k > 0:
    query_vec=values/k
else: 
    query_vec=values

query_vec = pmean.z_norm(query_vec)
#normlize query vector
query_vec = np.squeeze(normalize(query_vec.reshape(1,-1)))

In [16]:
np.shape(w2v.syn0)

/home/domyoung/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(4040272, 300)

In [17]:

#calculate similiarity
sim = sent_matrix@query_vec
#sort the similiarty array
rank_idx=(sim).argsort()[::-1]

for i in range(10):
    print(idx2sent[rank_idx[i]],sim[rank_idx[i]])
    print("==")

 블루링크(Blue Link) 0.8749406490233367
==
 하이빔 보조(HBA) 시스템
 0.7234573968893304
==
 선바이저 램프
 0.6907384330557564
==
 스마트키 리마인더
 0.6902303909256371
==
 테일게이트 열림
 0.6879389538108731
==
 리모컨 키
 0.6878028519391455
==
 제동등을 점검하십시오
 0.6873355768935303
==
 와셔액 분출
 0.6834689171904679
==
 미등 표시등
 0.6833882003456752
==
 미등 위치
 0.6833882003456752
==


# word2vec p-mean

In [18]:
p_sent_matrix=np.empty((0,dim*len(meanlist)))
p_sent2idx={}
p_idx2sent={}

i=0

for line in line_list:
    values = np.zeros(dim)
    # indexing
    p_sent2idx[line]=i
    p_idx2sent[i]=line


    k=0
    if(" @ " in line):
        line=line[:line.index(" @ ")]
        #strip newline
        line = line.replace('\n','')
    words = line.split(' ')
    values = pmean.get_sentence_embedding(words,w2v,meanlist)
    i+=1
    p_sent_matrix=np.vstack((p_sent_matrix,pmeanFT.z_norm(values)))
    if(i%100000==0):
        print(datetime.datetime.now().isoformat())
p_sent_matrix = normalize(p_sent_matrix)

No word embeddings for sentence:
['스마트키\n']
No word embeddings for sentence:
['도어\n']
No word embeddings for sentence:
['도난경보장치\n']
No word embeddings for sentence:
['경음기\n']
No word embeddings for sentence:
['미러\n']
No word embeddings for sentence:
['실내미러\n']
No word embeddings for sentence:
['전동식\n']
No word embeddings for sentence:
['테일게이트\n']
No word embeddings for sentence:
['연료주입구\n']
No word embeddings for sentence:
['계기판\n']
No word embeddings for sentence:
['계측기\n']
No word embeddings for sentence:
['LCD표시창\n']
No word embeddings for sentence:
['empty\n']
No word embeddings for sentence:
['조명\n']
No word embeddings for sentence:
['와이퍼/와셔\n']
No word embeddings for sentence:
['실내등\n']
No word embeddings for sentence:
['맵램프\n']
No word embeddings for sentence:
['룸램프\n']
No word embeddings for sentence:
['퍼들', '램프\n']
No word embeddings for sentence:
['서리제거(열선)\n']


In [19]:
query = "리모컨 키 배터리 교체 방법 알려줘"
query_vec =None

words=query.split(' ')
query_vec = pmean.get_sentence_embedding(words,w2v,meanlist)
query_vec = pmean.z_norm(query_vec)
#normlize query vector
query_vec = np.squeeze(normalize(query_vec.reshape(1,-1)))

In [21]:

#calculate similiarity
sim = p_sent_matrix@query_vec
#sort the similiarty array
rank_idx=(sim).argsort()[::-1]

for i in range(10):
    print(p_idx2sent[rank_idx[i]],sim[rank_idx[i]])
    print("==")

리모컨 키 사용 시 주의사항
 0.8348408290224996
==
리모컨 키 사용 방법
 0.8309919034106776
==
 이모빌라이저 경고등 (리모컨 키 장착 차량)
 0.7742589778396123
==
건전지 교체 방법
 0.7606527566342907
==
 키 확인 표시등 (스마트키 장착 차량)
 0.7590550066325026
==
 배터리 방전 방지 기능(차량 조명을 켜 놓은채 실수로 하차시 조명 자동 소등)
 0.7533795175804063
==
 리모컨 키
 0.7328140734877977
==
 도어 잠금 해제(2)(세이프티 언락 기능 설정 시)
 0.7226928476374197
==
 스마트키 배터리 전압이 낮습니다
 0.7196564176536591
==
도어 잠금 해제 (세이프티 언락 기능 설정 시)
 0.7189516022424276
==


# FastText

In [27]:
fT_sent_matrix=np.empty((0,dim))
fT_sent2idx={}
fT_idx2sent={}

i=0

for line in line_list:

    # indexing
    fT_sent2idx[line]=i
    fT_idx2sent[i]=line

    #remove the "@ ~" part

    if(" @ " in line):
        line=line[:line.index(" @ ")]
    #strip newline
    line = line.replace('\n','')
    values = fT.get_sentence_vector(line)
    values = np.squeeze(normalize(pmeanFT.z_norm(values).reshape(1,-1)))

    fT_sent_matrix=np.vstack((fT_sent_matrix,values))
    i+=1
fT_sent_matrix = normalize(fT_sent_matrix)

## compute max sentence vector 


In [29]:
q_vec= pmeanFT.z_norm(fT.get_sentence_vector("리모컨 키 배터리 교체 방법 알려줘"))
q_vec = np.squeeze(normalize(q_vec.reshape(1,-1)))


In [31]:
fT_sim = fT_sent_matrix@q_vec
fT_rank_idx=(fT_sim).argsort()[::-1]
for i in range(10):
    print(fT_idx2sent[fT_rank_idx[i]],fT_sim[fT_rank_idx[i]])
    print("==")
    


리모컨 키 사용 방법
 0.858838675247939
==
건전지 교체 방법
 0.8363520237012205
==
리모컨 키
 0.802192856030285
==
 리모컨 키
 0.802192856030285
==
리모컨 키 사용 시 주의사항
 0.7801984878650643
==
 키 확인 표시등 (스마트키 장착 차량)
 0.7728797165619244
==
 스마트키 배터리 전압이 낮습니다
 0.7658651765612505
==
 이모빌라이저 경고등 (리모컨 키 장착 차량)
 0.7649227016925294
==
스마트키 버튼 사용 방법
 0.7605144123475809
==
 배터리 방전 방지 기능(차량 조명을 켜 놓은채 실수로 하차시 조명 자동 소등)
 0.7472516232548054
==


# Fasttext p-mean

In [32]:
pFT_sent_matrix=np.empty((0,dim*len(meanlist)))
pFT_sent2idx={}
pFT_idx2sent={}


i=0

for line in line_list:

    # indexing
    pFT_sent2idx[line]=i
    pFT_idx2sent[i]=line

    #remove the "@ ~" part

    if(" @ " in line):
        line=line[:line.index(" @ ")]
    #strip newlinea
    line = line.replace('\n','')

    words=line.split(" ") #split to words 

    sent_vec=pmeanFT.get_sentence_embedding(words, fT,meanlist) #make sentence vector

    pFT_sent_matrix=np.vstack((pFT_sent_matrix,sent_vec)) #add to sentence matrix 
    i+=1
pFT_sent_matrix = normalize(pFT_sent_matrix)

In [33]:
np.shape(pFT_sent_matrix)

(333, 900)

In [34]:
query="리모컨 키 건전지 교체 방법 알려줘"
words = query.split(" ")
pFT_query_vec=pmeanFT.z_norm(pmeanFT.get_sentence_embedding(words, fT,meanlist)) #make query vector
pFT_query_vec = np.squeeze(normalize(pFT_query_vec.reshape(1,-1)))

In [35]:
pFT_sim = pFT_sent_matrix@pFT_query_vec
pFT_rank_idx=(pFT_sim).argsort()[::-1]
for i in range(10):
    print(pFT_idx2sent[pFT_rank_idx[i]],pFT_sim[pFT_rank_idx[i]])
    print("==")

리모컨 키 사용 방법
 0.8875924427895896
==
건전지 교체 방법
 0.8806541115446684
==
리모컨 키 사용 시 주의사항
 0.8215448015331706
==
리모컨 키
 0.8121946960607308
==
 배터리 방전 방지 기능(차량 조명을 켜 놓은채 실수로 하차시 조명 자동 소등)
 0.8031932952039553
==
 키 확인 표시등 (스마트키 장착 차량)
 0.8008541206851127
==
 이모빌라이저 경고등 (리모컨 키 장착 차량)
 0.7939345125716173
==
파워 테일게이트 열림 높이 사용자 설정 방법
 0.7926408834087026
==
스마트키 버튼 사용 방법
 0.7918453720318595
==
스마트 테일게이트 작동 방법
 0.7830712934724449
==
